In [14]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas


In [15]:
df = pd.read_csv("pretest.csv",sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

In [17]:
df.head()

,visitor_id,date,email,experiment,group,submitted
0,1,2021-12-01,NaN,NaN,NaN,0
1,2,2021-12-01,NaN,NaN,NaN,0
2,3,2021-12-01,NaN,NaN,NaN,0
3,4,2021-12-01,NaN,NaN,NaN,0
4,5,2021-12-01,NaN,NaN,NaN,0


In [19]:
print("Number of Rows:", df.shape[0])
print("Date Range:", df.date.min(), "-" , df.date.max())

Number of Rows: 309903
Date Range: 2021-12-01 - 2021-12-31


In [22]:
#Null Rate per Column
df.isnull().mean()

visitor_id    0.000000
date          0.000000
email         0.899017
experiment    0.903070
group         0.903070
submitted     0.000000
dtype: float64